## Introduction

Research Question: How does the level of debt owed to the World Bank, intellectual property, and balance of payments correlate with the GDP growth rates of countries, and are there distinguishable patterns or differences in this relationship when comparing developed and developing nations?

Our project aims to research the relationship between a Country’s GDP and their debt to the World Bank Group and the International Monetary Fund. The World Bank’s mission is to “end extreme poverty and promote shared prosperity in a sustainable way” (World Bank Group, 2023). However, there is extreme controversy on whether their involvement in developing countries’ economic development is positive or exploitative (Nagle, 2019). Seeing whether there is a relationship between a country’s economic prosperity (measured by GDP) and the pattern of loans("Borrower's Obligation") received from these Multi-Governmental Organizations would provide an interesting insight into whether countries's economic livelihood is more negatively affected by the involvement of these MGOs. There is detailed and immensely available data that correlates to this topic because there is information on GDP and Debt for all countries, also broken down by developing versus developed countries. 


## Data Description
We have four data tables for this phase: The GDP data for countries around the world, Loans/debt owed to the World Bank by countries, a dataset containing a list of developed countries (as classified biy the UN), and the Intellectual Property of countries for years 2000 to 2022.

The GDP Dataset details the gross domestic product for individual countries across a set of years. Specifically, the dataset contains years from 1960 to 2022 with the respective GDPs - in USD dollars - for 267 countries around the world.

The loans/debt dataset discusses a given amount of money loaned to a country from the World Bank and what how that money has been handled since. This particular dataset contains 148 countries and their original principal amount,canceled amount, undisbursed amount, disbursed amount, and borrower's obligation.

For the intellectual property dataset it pertains to transactions between residents and non-residents concerning the authorized utilization of intellectual property rights. This encompasses rights like patents, trademarks, copyrights, industrial designs, trade secrets, and franchises. Additionally, it covers the usage of original productions or prototypes, which include copyrights on literature, software, films, and audio recordings, as well as associated rights for live events and various broadcast methods. The monetary values provided in the dataset are presented in contemporary U.S. dollars (World Bank, 2023). 

There is also a datset containing developed countries based on the United Nations classification. This is going to be used to compare if there are any patterns or trends with the dependent variables discussed previously and developing and developed nations.

Link to balance of payments of countries owed to World Bank: https://finances.worldbank.org/Loans-and-Credits/IBRD-Country-wise-Loan-summary/5xqk-t59j

Link to intellectual property: https://data.worldbank.org/indicator/BM.GSR.ROYL.CD?locations=SV

Link to GDP: https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?end=2022&start=2022&view=bar

Link to Developed Countries Dataset: https://worldpopulationreview.com/country-rankings/developed-countries


## Preregistration of Hypotheses

Hypothesis 1: If a country’s status is developing, then they will have greater borrower’s obligation compared to a developed country. 

Analysis: We will conduct a logistic regression where we input whether or not a country is developed or developing (a dummy variable) and output the borrower’s obligation (in USD dollars). The developed country dummy will be the reference variable, so we will test whether e, where  is the coefficient output by the regression equation for the ‘developing’ variable, is greater than 0.

Hypothesis 2: The higher a country’s GDP (in USD Dollars), the higher the intellectual property charges (in USD Dollars). 

Analysis: We will conduct a linear regression where we input the GDP of a country and output the intellectual property charges. We would test whether  (the regression coefficient of GDP in our equation) is greater than 0 (if an increase in GDP causes an increase in Intellectual Property Charges). 


## Data Analysis 

## Evaluation of Significance

## Intepretation and Conclusions

## Limitations